# Cell for testing

Use this cell to test temporary code.

In [0]:
import os



# Setup

## Main assumptions made:
- We do not have access to whether the product has sold (obviously)
- We do NOT have access to current_sale_amount. 

### Descriptions of variables from observation (PDF does not specify)

- `current_total_landing_count`	 = Total landed pages
- `current_other_landing_count`	= Total pages not involving product/landing
- `current_product_landing_count` = 	Total landed product pages
- `current_cart_landing_count`	= Total landed cart pages
- `current_sale_amount`	= TOTAL SOLD
- `current_is_sale`	= PRODUCT SOLD
- `current_avg_cart_amount` = 	Average cart money
- `current_avg_visited_product_price` =	Average product price
- `referrer` =	How was the site found
- `last_1_day_session_count`	= How many times did the user enter in the last day
- `last_7_day_session_count`	= How many times did the user enter in the last week
- `date` = What is the date during the session

# Initialize Google colab

Run code for Google Drive access.

In [0]:
# Import

from IPython.display import clear_output

# https://medium.com/@burakteke/tutorial-on-using-google-colab-for-kaggle-competition-620393c22821

# This is not necessary if already downloading from online source.

# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# Create a directory and mount Google Drive using that directory.
!mkdir -p my_drive
!google-drive-ocamlfuse my_drive

#!ls ~/my_drive

import os, os.path
os.chdir('/content/my_drive/Insider. - Likelihood/')

clear_output()

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18298 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0

Setting up unattended-upgrades (0.98ubuntu1.1) ...

Creating config file /etc/apt/apt.conf.d/20auto-upgrades with new version

Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up dirmngr (2.1.15-1ubuntu8) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) ...
Setting up python3-software-properties (0.96.24.17)

# Prepare the data

Read the likelihood csv file, convert to pandas.

Create seperate sale and fail datafiles for analysis.

Seperate training labels from features.

In [0]:
# (RUN) Importing, reading

# For classifier
from sklearn.ensemble import RandomForestRegressor
# For classification score
from sklearn.metrics import roc_auc_score
# For scoring accuracy of classification
from sklearn.metrics import accuracy_score
# For evaulating the Mean Squared Error of the model's scoring
from sklearn.metrics import mean_squared_error

import pandas as pd

# Read from csv
X = pd.read_csv("Likelihood_to_puchase_sample_data.csv", engine="python")

# Seperate sales from fails to describe and analyze later on.
sale = X[X['current_is_sale']==1]
fail = X[X['current_is_sale']==0]

# Drop the sale state from X, load to y for training.
y = X.pop("current_is_sale")

In [9]:
from IPython.display import display

print("Fail")
display(fail.head())
display(fail.describe()) # Describe the sessions which have not sold

print("Sale")
display(sale.head())
display(sale.describe())

Fail


,current_other_landing_count,current_product_landing_count,current_cart_landing_count,current_is_sale,current_avg_cart_amount,current_avg_visited_product_price,last_7_day_session_count,referrer_facebook,referrer_google
0,1,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,3,0,1
2,1,0,0,0,0,0,1,0,1
3,2,0,0,0,0,0,7,1,0
4,0,1,0,0,69,0,13,0,0


,current_other_landing_count,current_product_landing_count,current_cart_landing_count,current_is_sale,current_avg_cart_amount,current_avg_visited_product_price,last_7_day_session_count,referrer_facebook,referrer_google
count,856500.000000,856500.000000,856500.000000,856500.0,8.565000e+05,856500.000000,856500.000000,856500.000000,856500.000000
mean,2.342592,1.377630,0.081980,0.0,1.335948e+02,24.197647,93.561770,0.102964,0.287790
std,4.939385,3.509799,0.905087,0.0,8.590686e+04,36.280884,748.772775,0.303913,0.452733
min,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.0,0.000000e+00,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,0.000000,0.0,0.000000e+00,0.000000,1.000000,0.000000,0.000000
75%,2.000000,1.000000,0.000000,0.0,0.000000e+00,49.000000,5.000000,0.000000,1.000000
max,395.000000,276.000000,287.000000,0.0,7.928758e+07,547.000000,7006.000000,1.000000,1.000000


Sale


,current_other_landing_count,current_product_landing_count,current_cart_landing_count,current_is_sale,current_avg_cart_amount,current_avg_visited_product_price,last_7_day_session_count,referrer_facebook,referrer_google
158,3,2,2,1,74,87,19,0,0
365,13,5,6,1,55,47,2,0,0
720,9,4,3,1,58,99,6,0,1
768,15,11,21,1,155,84,11,0,0
825,2,2,3,1,134,0,0,0,1


,current_other_landing_count,current_product_landing_count,current_cart_landing_count,current_is_sale,current_avg_cart_amount,current_avg_visited_product_price,last_7_day_session_count,referrer_facebook,referrer_google
count,4934.000000,4934.000000,4934.000000,4934.0,4934.000000,4934.000000,4934.000000,4934.000000,4934.000000
mean,10.439400,7.628091,4.024524,1.0,89.521889,42.270166,117.568504,0.088366,0.304418
std,15.269372,12.136184,6.591710,0.0,96.529689,33.576671,841.808283,0.283856,0.460207
min,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,1.0,30.250000,11.000000,0.000000,0.000000,0.000000
50%,5.000000,3.000000,2.000000,1.0,64.500000,42.500000,2.000000,0.000000,0.000000
75%,13.000000,9.000000,4.750000,1.0,116.000000,68.000000,7.000000,0.000000,1.000000
max,207.000000,167.000000,122.000000,1.0,1655.000000,138.000000,6992.000000,1.000000,1.000000


# Analysis of the general numerical data

---
### Observations

- `current_total_landing_count` mean is much higher than the failures by x6.
 - `current_other_landing_count` mean is higher by x5
 - `current_product_landing_count` mean is higher by x5.5
 - `current_cart_landing_count` mean is higher by a whopping x50!
- `current_sale_amount` mean is 0 if fail, above 0 if sale. Obvious bias.
- `current_avg_cart_amount` mean is interestingly lower by x0.5
- `current_avg_visited_product_price` mean is higher by x2
- `last_1_day_session_count` mean is higher only by x1.2
- `last_7_day_session_count` mean is similar: x1.25

`current_total_landing_count` is hierarchically the upper category which encompasses the other 3 landing counts. Hence, to prevent bias it is to be removed. **Check what the titanic dataset ahmed beysad(?) did for high category information.**

`current_sale_amount` mean is basically above 0 when a sale is made. Infinite bias, must be removed.


---
### Hypotheses

`current_cart_landing count` is more important than the other two landing subcategories, as it could be expected that those who are willing to buy are naturally going to ponder more on the cart page, returning again and again.)


`current_avg_cart_amount` could be lower as the customer is most likely searching for a specific application, and can only afford so much.



`current_avg_visited_product_price`

# Prepare features

Dummify string information and replace.

`referrer_other` dropped to avoid the dummy variable trap.

Index information dropped, assumed arbitrary.

`date` dropped until effective **day time parsing comes to fruition.**

`current_sale_amount` dropped.



In [0]:
def prep_features(X):
  # Referrers are dummified to get numerical information.
  referrer_dummies = pd.get_dummies(X['referrer'],prefix='referrer')
  X_refer = X.copy()
  X_refer = pd.concat([X,referrer_dummies],axis=1)
  # original refferer variable dropped since it is replaced.
  X_refer.drop('referrer',axis=1,inplace=True)
  # referrer_other dropped to avoid the dummy variable trap.
  X_refer.drop('referrer_other',axis=1,inplace=True)
  X = X_refer

  X.drop(X.columns[0],axis=1,inplace=True) # dropped as index information
  X.drop("date",axis=1,inplace=True) # dropped until effective day time parsing
  X.drop("current_sale_amount", 1, inplace=True)
  X.drop("last_1_day_session_count", 1, inplace=True)
  X.drop("current_total_landing_count", 1, inplace=True)
  return X

X = prep_features(X)

X.head()

sale = prep_features(sale)
fail = prep_features(fail)


#X = X[['last_1_day_session_count', 'referrer_google']].copy()

In [60]:
both = pd.concat([sale.iloc[:4934], fail.iloc[:20000]], ignore_index=True)

both.describe()
both.head(10)

AttributeError: ignored

In [0]:
# Get index of sales as list
# Use list to get the X's of each sold item.

both_y = both.pop("current_is_sale")

In [0]:
import numpy as np
from random import randint
from sklearn.preprocessing import MinMaxScaler

# (RUN) Scale
sc = MinMaxScaler()
sc.fit(X)
X = sc.transform(X)
sc.fit(both)
both = sc.transform(both)
#validate = sc.transform(validate)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    both, both_y, test_size=0.1, random_state=666, shuffle=True)

valid_set = (X_test, y_test)

In [55]:
y_train.mean()

0.19732620320855615

In [0]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

model = Sequential([
    Dense(16, input_shape=(8,), activation="relu"),
    Dense(32, activation="relu"),
    Dense(2, activation="softmax")
])

model.compile(Adam(lr=0.0001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [62]:
model.fit(X_train, y_train, validation_data=valid_set, batch_size=10, epochs=10, shuffle=True,)

Train on 22440 samples, validate on 2494 samples
Epoch 1/10
22440/22440 [==============================] - 14s 614us/step - loss: 0.2684 - acc: 0.8873 - val_loss: 0.2501 - val_acc: 0.8925
Epoch 2/10
20330/22440 [==========================>...] - ETA: 1s - loss: 0.2608 - acc: 0.8901

22440/22440 [==============================] - 13s 601us/step - loss: 0.2598 - acc: 0.8905 - val_loss: 0.2409 - val_acc: 0.8937
Epoch 3/10
22440/22440 [==============================] - 14s 615us/step - loss: 0.2516 - acc: 0.8949 - val_loss: 0.2325 - val_acc: 0.8966
Epoch 4/10
 9900/22440 [============>.................] - ETA: 7s - loss: 0.2473 - acc: 0.8961

22440/22440 [==============================] - 14s 616us/step - loss: 0.2438 - acc: 0.8985 - val_loss: 0.2247 - val_acc: 0.8990
Epoch 5/10
22440/22440 [==============================] - 14s 615us/step - loss: 0.2366 - acc: 0.9022 - val_loss: 0.2177 - val_acc: 0.9030
Epoch 6/10
 7750/22440 [=========>....................] - ETA: 8s - loss: 0.2202 - acc: 0.9115

22440/22440 [==============================] - 14s 603us/step - loss: 0.2300 - acc: 0.9047 - val_loss: 0.2113 - val_acc: 0.9078
Epoch 7/10
22440/22440 [==============================] - 14s 606us/step - loss: 0.2236 - acc: 0.9086 - val_loss: 0.2049 - val_acc: 0.9130
Epoch 8/10
 7590/22440 [=========>....................] - ETA: 8s - loss: 0.2187 - acc: 0.9095

22440/22440 [==============================] - 13s 598us/step - loss: 0.2177 - acc: 0.9123 - val_loss: 0.1998 - val_acc: 0.9126
Epoch 9/10
22440/22440 [==============================] - 14s 606us/step - loss: 0.2122 - acc: 0.9150 - val_loss: 0.1935 - val_acc: 0.9182
Epoch 10/10
 7550/22440 [=========>....................] - ETA: 8s - loss: 0.2101 - acc: 0.9177

22440/22440 [==============================] - 14s 608us/step - loss: 0.2072 - acc: 0.9196 - val_loss: 0.1883 - val_acc: 0.9194


In [0]:
from sklearn.metrics import roc_auc_score, accuracy_score

validate = fail.iloc[30000:]

validate_y = validate.pop("current_is_sale")
sc.fit(validate)
both = sc.transform(validate)
predicted = model.predict(validate)
predicted_classes = predicted.argmax(axis=-1)

validate_y = np.array(validate_y)
validate_y[0] = 1
print("validate accuracy", roc_auc_score(validate_y,predicted_classes))



In [67]:
predicted = model.predict(X_test)
predicted_classes = predicted.argmax(axis=-1)


print("validate set roc score", roc_auc_score(y_test, predicted_classes))

validate set roc score 0.8603140582626192


In [72]:
predicted_classes.mean()
validate_y.mean()

1.2099213551119177e-06

In [0]:
# (RUN) Scale

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X)
X = sc.transform(X)
validate = sc.transform(validate)

In [0]:
#(RUN) Validate
from sklearn.model_selection import train_test_split

###### original n_estimate=100
#model = RandomForestRegressor(n_estimators=50, oob_score=True, random_state=42)

X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#model.fit(X, y)

In [0]:
# Plotting

import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))

# Plot data
plt.scatter(fail.iloc[:,0], fail.iloc[:,4], color='red', marker='o', label='fail')
plt.scatter(sale.iloc[:,0], sale.iloc[:,4], color='blue', marker='x', label='sale')

plt.xlabel('ID')
plt.ylabel('current cart landing count')

plt.legend(loc="upper left")
plt.show()

In [0]:
import numpy as np

def balanced_subsample(x,y,subsample_size=1.0):

    class_xs = []
    min_elems = None

    for yi in np.unique(y):
        elems = x[(y == yi)]
        class_xs.append((yi, elems))
        if min_elems == None or elems.shape[0] < min_elems:
            min_elems = elems.shape[0]

    use_elems = min_elems
    if subsample_size < 1:
        use_elems = int(min_elems*subsample_size)

    xs = []
    ys = []

    for ci,this_xs in class_xs:
        if len(this_xs) > use_elems:
            np.random.shuffle(this_xs)

        x_ = this_xs[:use_elems]
        y_ = np.empty(use_elems)
        y_.fill(ci)

        xs.append(x_)
        ys.append(y_)

    xs = np.concatenate(xs)
    ys = np.concatenate(ys)

    return xs,ys

X, y  = balanced_subsample(X,y)

a

array([[-0.46894628, -0.11331046, -0.09742333, ..., -0.12245102,
        -0.33854276, -0.6358229 ],
       [-0.27261169, -0.38759557, -0.09742333, ..., -0.12511829,
        -0.33854276, -0.6358229 ],
       [-0.27261169, -0.38759557, -0.09742333, ..., -0.12111738,
        -0.33854276, -0.6358229 ],
       ...,
       [ 0.12005748,  0.16097466,  0.83576401, ..., -0.12511829,
        -0.33854276, -0.6358229 ],
       [-0.46894628, -0.11331046,  0.83576401, ..., -0.11978375,
        -0.33854276, -0.6358229 ],
       [ 6.40276417, 10.58380911, 14.83357419, ..., -0.12511829,
        -0.33854276, -0.6358229 ]])

In [0]:
one = 0
zero = 0

for i in b:
    if i ==1:
        one += 1
    else:
        zero += 1
        
print(one,zero)

4907 4907


In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
pipe_lr = make_pipeline(StandardScaler(), PCA(n_components=2), LogisticRegression(random_state=1))
pipe_lr.fit(X, y)
predicted = pipe_lr.predict(validate)

print('Test Accuracy: %.3f' % pipe_lr.score(validate, predicted))

for i in range(6000):
    if predicted[i] > 0:
        print(i, predicted[i])
print("-------")
        
for i in range(6000):
    if validate_y[i] > 0:
        print(i, validate_y[i])

print("-------")
        
for i in range(6000):
    if validate_y[i] > 0 & predicted[i] > 0:
        print(i, validate_y[i])
        
        
recall_score(validate_y,predicted)

print(accuracy_score(validate_y, predicted))
print(accuracy_score(validate_y, np.zeros(6000)))

In [0]:
import numpy as np

# Let's establish a baseline
output = np.zeros((861412,), dtype=int)
accuracy_score(validate_y,output,normalize=True)
mean_squared_error(validate_y, output)

# accuracy = 0.9942733558390178
# mean_squared_error = 0.0.005726644160982201

# We need to be better than these two numbers, otherwise we might as well
# say there were no sales and be accurate 99% of the time.

# The goal is to see what the real factors are.

# What we require is ANOMALY DETECTION, since (simply put) the occurence of
# a sale is very rare compared to the obvious non-sale.

# There could be many factors in someone not buying a product.
# Maybe they are still thinking about whether to buy it.
# Maybe they are simply browsing through. Maybe it's just a random visitor
# browsing out of curiosity or for the lolz. 

# There are also many factors in someone buying a product which might
# not be a feature in our dataset. Maybe it is the same guy who has been
# browsing on a different account and decided to buy the product with a 
# different account.

# Nevertheless, the main goal now is to find a way to differentiate this
# "anomaly" from the usual case of no sale occuring.
# This will be done through supervised classification via methods such as:
# - class_weight='balanced'
# - resample function
# - sythnetic sampling

In [0]:
# VERY BASIC CURRENT CART AVERAGE AMOUNT OVER 50 IF STATEMENT

data = pd.read_csv("C:/Users/HP/Desktop/Likelihood_to_puchase_sample_data.csv", engine="python")

above_50 = 0
is_sale = 0

# for i in range(6000):
#     if data.iloc[i, 4] > 50:
#         print(i, predicted.iloc[i])
#         above_50 += 1
#         if data.iloc[i,6] == 1:
#             is_sale += 1
            

#print(is_sale, above_50, is_sale / above_50)


# roc_auc_score(validate_y,)

above_50 = data[data['current_cart_landing_count']>=50]

above_50.head(200)

,Unnamed: 0,current_total_landing_count,current_other_landing_count,current_product_landing_count,current_cart_landing_count,current_sale_amount,current_is_sale,current_avg_cart_amount,current_avg_visited_product_price,referrer,last_1_day_session_count,last_7_day_session_count,date
2527,2528,404,198,131,72,0,0,324,13,other,0,3,15/05/2014 14:43
34379,34380,216,84,58,66,0,0,164,27,google,0,0,10/05/2014 22:45
88273,88274,371,86,210,71,0,0,887,34,other,0,0,18/05/2014 11:27
93507,93508,252,121,64,50,0,0,289,46,google,0,0,18/05/2014 01:45
101162,101163,500,70,131,287,0,0,168,7,other,2,5,15/05/2014 21:00
117053,117054,588,192,202,78,0,0,155,28,other,0,1,09/05/2014 14:49
141741,141742,281,175,34,63,0,0,83,37,other,4,10,11/05/2014 10:58
157962,157963,348,98,126,122,190,1,107,26,google,0,2,09/05/2014 05:42
167462,167463,139,56,15,67,133,1,108,25,other,0,3,14/05/2014 02:20
233804,233805,302,117,120,61,0,0,555,60,other,2,23,07/05/2014 10:35


In [0]:
# TRY MULTIPLE MODELS

from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

train = X
targets = y

clf = RandomForestClassifier(n_estimators=50, max_features='sqrt')
clf = clf.fit(train, targets)

model = SelectFromModel(clf, prefit=True)
train_reduced = model.transform(train)
print(train_reduced.shape)

validate_reduced = model.transform(validate)
print(validate_reduced.shape)

def compute_score(clf, X, y, scoring='roc_auc'):
    xval = cross_val_score(clf, X, y, cv = 5, scoring=scoring)
    return np.mean(xval)

logreg = LogisticRegression()
logreg_cv = LogisticRegressionCV()
rf = RandomForestClassifier()
gboost = GradientBoostingClassifier()

models = [logreg, logreg_cv, rf, gboost]

for model in models:
    print('Cross-validation of : {0}'.format(model.__class__))
    score = compute_score(clf=model, X=validate_reduced, y=validate, scoring='roc_auc')
    print('CV score = {0}'.format(score))
    print('****')
    
parameters = {'bootstrap': False, 'min_samples_leaf': 3, 'n_estimators': 50, 
              'min_samples_split': 10, 'max_features': 'sqrt', 'max_depth': 6}

model = RandomForestClassifier(**parameters)
model.fit(train, targets)

predicted=model.predict(validate)

roc_auc_score(validate_y,predicted)

In [0]:
predicted = np.zeros(6000)

for i in range(6000):
    if data.iloc[i,1] > 2 and data.iloc[i,2] > 2 and data.iloc[i,3] > 2 and data.iloc[i,4] > 5 and data.iloc[i,8] > 2 and data.iloc[i,11] > 0:
        predicted[i] = 1

# for i in range(6000):
#     if data.iloc[i, 4] > 50:
#         print(i, predicted.iloc[i])
#         above_50 += 1
#         if data.iloc[i,6] == 1:
#             is_sale += 1

for i in range(6000):
    if predicted[i] > 0:
        print(i, predicted[i])
print("-------")
        
for i in range(6000):
    if validate_y[i] > 0:
        print(i, validate_y[i])

print("-------")
        
for i in range(6000):
    if validate_y[i] > 0 and predicted[i] > 0:
        print(i, validate_y[i])
        
        
recall_score(validate_y,predicted)

print(accuracy_score(validate_y, predicted))
print(accuracy_score(validate_y, np.zeros(6000)))


365 1.0
642 1.0
768 1.0
1603 1.0
1940 1.0
2527 1.0
2795 1.0
2836 1.0
2846 1.0
2914 1.0
3378 1.0
3696 1.0
4601 1.0
4647 1.0
5844 1.0
-------
158 1
365 1
720 1
768 1
825 1
889 1
950 1
1069 1
1280 1
1323 1
1789 1
2329 1
2468 1
2690 1
2914 1
2974 1
3696 1
3720 1
3918 1
4109 1
4125 1
4201 1
4601 1
4712 1
4818 1
5174 1
5798 1
-------
365 1
768 1
2914 1
3696 1
4601 1
0.9946666666666667
0.9955


In [0]:
from sklearn import svm

model = svm.SVC(class_weight='balanced', probability=False,random_state=1)

model.fit(X,y)

predicted = model.predict(validate)

for i in range(6000):
    if predicted[i] > 0:
        print(i, predicted[i])
print("-------")
        
for i in range(6000):
    if validate_y[i] > 0:
        print(i, validate_y[i])

print("-------")
        
for i in range(6000):
    if validate_y[i] > 0 & predicted[i] > 0:
        print(i, validate_y[i])
        
        
recall_score(validate_y,predicted)

print(accuracy_score(validate_y, predicted))
print(accuracy_score(validate_y, np.zeros(6000)))

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
import numpy as np

model = RandomForestClassifier(random_state=1, n_estimators = 100, n_jobs=-1)

model.fit(X,y)

predicted = model.predict(validate)

for i in range(6000):
    if predicted[i] > 0:
        print(i, predicted[i])
print("-------")
        
for i in range(6000):
    if validate_y[i] > 0:
        print(i, validate_y[i])

print("-------")
        
for i in range(6000):
    if validate_y[i] > 0 and predicted[i] > 0:
        print(i, validate_y[i])
        
        
recall_score(validate_y,predicted)

print(accuracy_score(validate_y, predicted))
print(accuracy_score(validate_y, np.zeros(6000)))

6 1.0
9 1.0
52 1.0
58 1.0
70 1.0
89 1.0
106 1.0
108 1.0
111 1.0
116 1.0
120 1.0
129 1.0
136 1.0
155 1.0
158 1.0
159 1.0
173 1.0
178 1.0
202 1.0
212 1.0
221 1.0
223 1.0
228 1.0
291 1.0
297 1.0
326 1.0
349 1.0
354 1.0
365 1.0
394 1.0
401 1.0
402 1.0
403 1.0
404 1.0
418 1.0
436 1.0
445 1.0
448 1.0
452 1.0
463 1.0
466 1.0
502 1.0
547 1.0
557 1.0
575 1.0
583 1.0
599 1.0
630 1.0
636 1.0
642 1.0
655 1.0
683 1.0
720 1.0
726 1.0
743 1.0
749 1.0
752 1.0
754 1.0
768 1.0
774 1.0
782 1.0
784 1.0
799 1.0
811 1.0
821 1.0
825 1.0
853 1.0
857 1.0
859 1.0
875 1.0
877 1.0
889 1.0
900 1.0
918 1.0
930 1.0
950 1.0
951 1.0
978 1.0
997 1.0
1010 1.0
1014 1.0
1050 1.0
1053 1.0
1054 1.0
1061 1.0
1066 1.0
1069 1.0
1085 1.0
1087 1.0
1094 1.0
1098 1.0
1136 1.0
1140 1.0
1163 1.0
1165 1.0
1174 1.0
1180 1.0
1214 1.0
1226 1.0
1239 1.0
1242 1.0
1245 1.0
1269 1.0
1271 1.0
1277 1.0
1280 1.0
1302 1.0
1310 1.0
1314 1.0
1322 1.0
1323 1.0
1336 1.0
1352 1.0
1356 1.0
1377 1.0
1389 1.0
1400 1.0
1403 1.0
1407 1.0
1413 1.0
1417 1.

In [0]:
print(validate.iloc[2690], "\n")
print(validate.iloc[52])

In [0]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X, y)

predicted = clf.predict(validate)

for i in range(6000):
    if predicted[i] > 0:
        print(i, predicted[i])
print("-------")
        
for i in range(6000):
    if validate_y[i] > 0:
        print(i, validate_y[i])

print("-------")
        
for i in range(6000):
    if validate_y[i] > 0 & predicted[i] > 0:
        print(i, validate_y[i])

print(recall_score(validate_y,predicted))

print(accuracy_score(validate_y, predicted))
print(accuracy_score(validate_y, np.zeros(6000)))

In [0]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(class_weight="balanced", random_state=0, n_jobs=-1)

clf.fit(X, y)

predicted = clf.predict(validate)

for i in range(6000):
    if predicted[i] > 0:
        print(i, predicted[i])
print("-------")
        
for i in range(6000):
    if validate_y[i] > 0:
        print(i, validate_y[i])

print("-------")
        
for i in range(6000):
    if validate_y[i] > 0 & predicted[i] > 0:
        print(i, validate_y[i])

print(recall_score(validate_y,predicted))

print(accuracy_score(validate_y, predicted))
print(accuracy_score(validate_y, np.zeros(6000)))

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


9 1
52 1
106 1
108 1
116 1
120 1
129 1
158 1
173 1
178 1
223 1
349 1
354 1
365 1
401 1
418 1
445 1
448 1
452 1
463 1
547 1
557 1
575 1
630 1
642 1
655 1
683 1
720 1
743 1
752 1
768 1
774 1
811 1
825 1
859 1
875 1
877 1
889 1
950 1
978 1
997 1
1014 1
1066 1
1069 1
1085 1
1087 1
1094 1
1136 1
1140 1
1176 1
1180 1
1226 1
1245 1
1250 1
1271 1
1280 1
1302 1
1322 1
1323 1
1336 1
1352 1
1377 1
1400 1
1403 1
1407 1
1413 1
1423 1
1437 1
1445 1
1452 1
1460 1
1499 1
1509 1
1515 1
1600 1
1603 1
1606 1
1665 1
1667 1
1711 1
1724 1
1725 1
1775 1
1789 1
1802 1
1807 1
1808 1
1830 1
1831 1
1890 1
1929 1
1937 1
1940 1
2003 1
2059 1
2085 1
2101 1
2133 1
2137 1
2154 1
2224 1
2301 1
2329 1
2340 1
2360 1
2386 1
2418 1
2440 1
2443 1
2468 1
2469 1
2527 1
2587 1
2591 1
2607 1
2613 1
2616 1
2684 1
2690 1
2722 1
2726 1
2729 1
2737 1
2755 1
2795 1
2836 1
2846 1
2858 1
2914 1
2917 1
2932 1
2956 1
2974 1
2975 1
2979 1
2990 1
3010 1
3070 1
3082 1
3095 1
3107 1
3117 1
3127 1
3135 1
3152 1
3166 1
3182 1
3188 1
3203 1
3

In [0]:
plt.legend([a.collections[0], b.collections[0]], ["non weighted", "weighted"],
           loc="upper right")
plt.show()

X.iloc[:20,:2]

In [0]:
from sklearn import svm
model = svm.SVC(class_weight='balanced', probability=True)#kernel='linear')#,c=1,gamma=1)

model.fit(X.iloc[:5000],y.iloc[:5000])

model.score(X,y)

predicted=model.predict(validate)

predicted

roc_auc_score(validate_y,predicted)

In [0]:
from matplotlib.colors import ListedColormap
import numpy as np


def plot_decision_regions(X, y, classifier, resolution=0.02):

    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # plot class samples
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
                    alpha=0.8, c=cmap(idx),
                    edgecolor='black',
                    marker=markers[idx], 
                    label=cl)
        
plot_decision_regions(X,y, classifier=svm)
plt.show()

In [0]:
from sklearn import svm
model = svm.SVC(class_weight='balanced', probability=True)#kernel='linear')#,c=1,gamma=1)

model.fit(X.iloc[:10000],y.iloc[:10000])

model.score(X,y)

predicted=model.predict(validate)

predicted

In [0]:
from sklearn import svm

model = svm.SVC(class_weight='balanced', probability=False,random_state=1)#kernel='linear')#,c=1,gamma=1)

X = pd.concat([sale.iloc[:4000], fail.iloc[:4000]])
y = X.pop("current_is_sale")

validate_X = pd.concat([sale.iloc[4000:4900], fail.iloc[4000:4900]])
validate_y = validate_X.pop("current_is_sale")

# Referrers are dummified to get numerical information.
referrer_dummies = pd.get_dummies(X['referrer'],prefix='referrer')
X_refer = X.copy()
X_refer = pd.concat([X,referrer_dummies],axis=1)
# original refferer variable dropped since it is replaced.
X_refer.drop('referrer',axis=1,inplace=True)
# referrer_other dropped to avoid the infamous dummy variable trap.
X_refer.drop('referrer_other',axis=1,inplace=True)
X = X_refer

X.drop(X.columns[0],axis=1,inplace=True) # dropped as index information
X.drop("date",axis=1,inplace=True) # dropped until effective day time parsing
X.drop("current_sale_amount", 1, inplace=True)


# Referrers are dummified to get numerical information.
referrer_dummies = pd.get_dummies(validate_X['referrer'],prefix='referrer')
X_refer = validate_X.copy()
X_refer = pd.concat([validate_X,referrer_dummies],axis=1)
# original refferer variable dropped since it is replaced.
X_refer.drop('referrer',axis=1,inplace=True)
# referrer_other dropped to avoid the infamous dummy variable trap.
X_refer.drop('referrer_other',axis=1,inplace=True)
validate_X = X_refer

validate_X.drop(X.columns[0],axis=1,inplace=True) # dropped as index information
validate_X.drop("date",axis=1,inplace=True) # dropped until effective day time parsing
validate_X.drop("current_sale_amount", 1, inplace=True)

#TEST
#X = X[['last_1_day_session_count', 'referrer_google']].copy()

# validate = X.iloc[4000:].copy()
# validate_y = y.iloc[4000:].copy()
# X = X.iloc[:4000].copy()
# y = y.iloc[:4000].copy()


model.fit(X,y)

model.score(X,y)

predicted=model.predict(validate_X)

for i in range(1800):
    if predicted[i] > 0:
        print(i, predicted[i])
print("-------")
        
for i in range(1800):
    if validate_y[i] > 0:
        print(i, validate_y[i])

print("-------")
        
for i in range(1800):
    if validate_y[i] > 0 & predicted[i] > 0:
        print(i, validate_y[i])

print(recall_score(validate_y,predicted))

print(accuracy_score(validate_y, predicted))
print(accuracy_score(validate_y, np.zeros(1800)))


In [0]:
print(predicted.shape)
import numpy as np
#zero = np.zeros(6000,)


for i in range(6000):
    if predicted[i] > 0:
        print(i, predicted[i])
#accuracy_score(validate_y,predicted,normalize=True)

print("-------")

for i in range(6000):
    if validate_y[i] > 0:
        print(i, validate_y[i])


roc_auc_score(validate_y,predicted)
roc_auc_score(validate_y,zero)
#predicted.show()

In [0]:
import matplotlib.pyplot as plt

# Plot data
plt.scatter(fail.iloc[0:4900,0], fail.iloc[:4900,1], color='red', marker='o', label='fail',s=1)

#x = 
#y = 
#heatmap, xedges, yedges = np.histogram2d(x,y,bins=50)
#extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.xlabel('Current cart landing count')
plt.ylabel('Current average cart amount')

plt.legend(loc="upper left")

#plt.clf()
#plt.imshow(heatmap.T, extent, origin='lower')

plt.show()

In [0]:
plt.scatter(sale.iloc[0:4900,0], sale.iloc[0:4900,1], color='blue', marker='x', label='sale',s=1)

plt.xlabel('Current cart landing count')
plt.ylabel('Current average cart amount')

plt.legend(loc="upper left")

#plt.clf()
#plt.imshow(heatmap.T, extent, origin='lower')

plt.show()

In [0]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import recall_score

clf = MLPClassifier(random_state=1)
clf.fit(X, y)

predicted = clf.predict(validate)

recall_score(validate_y, predicted)

In [0]:
## UNFINISHED
@staticmethod
def totalWeights(nInput, nHidden, nOutput):
    tw = (nInput * nHidden) + (nHidden * nOutput) + nHidden + nOutput
    return tw

def main():
    global X
    global y
    thedata = np.loadtxt(X, delime

In [0]:
## TOO LONG WAIT
## TODO: TRY FOR SMALLER TEST!
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# instantiate K (k = 2)
knn = KNeighborsClassifier(n_jobs=-1)

# fitting the model
knn.fit(X[:100000],y[:100000])
#knn.fit(X,y)

# predict the response
y_predict = knn.predict(validate)

# evaluate accuracy
print("Accuracy of 1NN: %.2f %%" %(100*accuracy_score(validate_y, y_predict)))

Accuracy of 1NN: 99.47 %


In [0]:
k = 0
for i in y_predict:
    k+= 1
    if i > 0:
        print(k,i)

print("---")
        
for i in range(6000):
    if validate_y[i] > 0:
        print(i, validate_y[i])

464 1
643 1
826 1
1438 1
3428 1
4246 1
4996 1
---
158 1
365 1
720 1
768 1
825 1
889 1
950 1
1069 1
1280 1
1323 1
1789 1
2329 1
2468 1
2690 1
2914 1
2974 1
3696 1
3720 1
3918 1
4109 1
4125 1
4201 1
4601 1
4712 1
4818 1
5174 1
5798 1


In [0]:
y_oob = model.oob_prediction_
print("c-stat: ", roc_auc_score(y, y_oob))

In [0]:
features = pd.DataFrame()
features['feature'] = X.columns
features['importance'] = model.feature_importances_
features.sort_values(by=['importance'], ascending=True, inplace=True)
features.set_index('feature', inplace=True)


features.plot(kind='barh', figsize=(20, 20))